In [150]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv("../csv/data.csv", header=None)
df.columns = ('category', 'text', 'postid')

In [151]:
df.head()

,category,text,postid
0,general labor,FERRARA CANDY *** PACKAGING ASSISTANTS *** Dek...,7405032484
1,customer service,Office Customer Service Part - Time 24 hrs/wk ...,7405099747
2,resumes / job wanted,quality tech control (west burbs) Manufacturin...,7396174532
3,resumes / job wanted,Driver Job Wanted (Chicago) CDL-Class-B Lookin...,7400075704
4,transportation,$400/DAY Class A LOCAL/REGIONAL CDL Driver (Le...,7406024714


In [152]:
id = 1
seen = {}

for idx, row in df.iterrows():
    if row.category == 'resumes / job wanted':
        df.at[idx, 'category'] = 0
    else:
        if row.category in seen:
            df.at[idx, 'category'] = seen[row.category]
        else:
            df.at[idx, 'category'] = id
            seen[row.category] = id
            id += 1

In [153]:
import nltk
# import pandas as pd
# import csv
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [154]:
job_list = df[df["category"] != 0]
resume_list = df[df["category"] == 0]

In [155]:
processed_collection_r = []
processed_collection_j = []
lemmatizer = nltk.stem.WordNetLemmatizer()

for post in resume_list.text:
    #tokenize each review
    tokens = nltk.word_tokenize(post)

    #lemmatize each review and convert it to lower
    tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens]

    #Remove Stop Words and Punctuations
    tokens = [token for token in tokens if not token in stopwords.words('english') if token.isalpha()]

    joins = " ".join(tokens)
    processed_collection_r.append(joins)

for post in job_list.text:
    #tokenize each review
    tokens = nltk.word_tokenize(post)

    #lemmatize each review and convert it to lower
    tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens]

    #Remove Stop Words and Punctuations
    tokens = [token for token in tokens if not token in stopwords.words('english') if token.isalpha()]

    joins = " ".join(tokens)
    processed_collection_j.append(joins)
    

In [156]:
X = processed_collection_j
y = list(job_list.category)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [157]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=1)
vectorizer.fit(x_train)
x_train_m = vectorizer.transform(x_train)
x_test_m = vectorizer.transform(x_test)

In [158]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [159]:
nb.fit(x_train_m, y_train)

MultinomialNB()

In [160]:
y_pred_nb = nb.predict(x_test_m)

In [161]:
from sklearn.metrics import accuracy_score
acc_nb = accuracy_score(y_test, y_pred_nb)
print("Accuracy: {}%".format(round(acc*100,2)))

Accuracy: 57.86%


In [162]:
from sklearn.svm import LinearSVC
svm = LinearSVC()

In [163]:
svm.fit(x_train_m, y_train)

LinearSVC()

In [164]:
y_pred_svm = svm.predict(x_test_m)

In [165]:
acc_svm = accuracy_score(y_test, y_pred_svm)
print("SVM Accuracy: {}%".format(round(acc_svm*100, 2)))

SVM Accuracy: 78.93%


In [166]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, max_depth=75, bootstrap=True, random_state=123)

rf.fit(x_train_m, y_train)
y_pred_rf = rf.predict(x_test_m)
acc_rf = accuracy_score(y_test, y_pred_rf)
print("RF Accuracy: {}%".format(round(acc_rf*100, 2)))

RF Accuracy: 70.71%


In [167]:
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(20,20,15,10), random_state=123)
nn.fit(x_train_m, y_train)
y_pred_nn = nn.predict(x_test_m)
acc_nn = accuracy_score(y_test, y_pred_nn)
print("NN Accuracy: {}%".format(round(acc_nn*100, 2)))

NN Accuracy: 66.55%


/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [168]:
resume_vec = vectorizer.transform(processed_collection_r)
y_resume = svm.predict(resume_vec)

In [169]:
new_df = pd.DataFrame(resume_list.text, y_resume)

In [170]:
categories = {v: k for k, v in seen.items()}

In [171]:
resume_list['pred_category'] = y_resume
resume_list.pred_category = resume_list.pred_category.astype('object')
for idx, row in resume_list.iterrows():
    resume_list.at[idx, 'pred_category'] = categories[row.pred_category]

resume_list.to_csv("predicted_categories.csv")

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5507: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [174]:
from sklearn.metrics import cosine_similarity
vectorizer5 = TfidfVectorizer()
vectorizer5.fit(processed_collection_j)
# print(vectorizer5.vocabulary_)
first_job = x_train_m[0]
first_job_y = y_train[0]
possible_matches = 

cosine_similarity([v,)
jaccard_score(v[0,:],v[1,:])

ImportError: cannot import name 'cosine_similarity' from 'sklearn.metrics' (/anaconda3/lib/python3.7/site-packages/sklearn/metrics/__init__.py)